Start from scratch.... let's draw from one random distribution some number of times.

Then, add it to the result of a second distribution. Store the results

Then, consider time phase components

When these modules are developed, its ready to start plugging it in to excel

In [1]:
import numpy as np
import pandas as pd
import scipy as sp

import matplotlib.pyplot as plt



import random

# Class only
To start: Add one value of several uniform variables

## Advantages:
Because wbs is calculated separately from random variables, the whole workbook doesn't need to be relcaculated for an update

In [3]:
master_years = [0, 1, 2, 3, 4]

In [88]:
simulatios = 20

In [155]:
class random_variable:
    def __init__(self, pe=None, distribution=None, time_phasing=None, mean=0, std=0, cv=None, lower_bound=None, upper_bound=None, draws_for_simulation=[], seed_column_number=None):
        
        """
        Distributions: 
            - Uniform: Takes upper_bound, lower_bound
        """
        
        self.pe = float(pe)
        self.distribution = distribution
        self.mean = mean
        self.std = std
        self.cv = cv
        self.upper_bound = upper_bound
        self.lower_bound = lower_bound
        self.draws_for_simulation = draws_for_simulation
        self.seed_column_number = seed_column_number
        self.time_phasing = time_phasing
    
    def time_phase(self, level_per_year):
        adj_time_phasing = []
        for year in range(len(master_years)-1):
            if year >= len(level_per_year):
                adj_time_phasing += [0]
            else:
                adj_time_phasing += level_per_year[year]
        self.time_phasing= np.array(adj_time_phasing)
                
    def get_draws(self, steps):
        if self.distribution == 'uniform':
            step_size = abs((self.upper_bound - self.lower_bound) / steps)
            return [x*step_size + self.lower_bound for x in range(steps)]
        else:
            print('   distribution not defined')
            
    def value(self):
        return self.pe

In [144]:
# Return basic list of variable values
def generate_values_table(list_of_variables, steps):
    df = pd.DataFrame()
    counter = 0
    for variable in list_of_variables:
        if variable.distribution is None:
            df["var_"+str(counter)] = [variable.pe for i in range(steps)]
        else:
            df["var_"+str(counter)] = variable.get_draws(steps=steps)
        variable.seed_column_number = counter
        counter +=1
    return df
#Return basic list of seeds - This is done explicitly to allow for correlation adjustment later
def generate_order_table(values_table):
    _df_seeds = pd.DataFrame()
    order_list = [i for i in range(values_table.shape[0])]
    for col in values_table.columns:
        random.shuffle(order_list)
        _df_seeds[col] = order_list
    return _df_seeds
# Function to sort values according to seed
def generate_draw_table(order_table, values_table):
    _df_draws = pd.DataFrame()
    
    for col in order_table.columns:
        _df_draws[col] = order_table[col].apply(lambda x: values_table[col].iloc[x])
    
    return _df_draws
# Condensed verion of above steps
def generate_shuffled_draws(list_of_variables, steps):
    df_raw_values = generate_values_table(list_of_variables=list_of_variables, steps=steps)
    df_seeds = generate_order_table(df_raw_values)
    _df_shuffled_draws =  generate_draw_table(df_seeds, df_raw_values)
    #Store risk values back into variable
    for i in range(len(df_raw_values.columns)):
        list_of_variables[i].draws_for_simulation = np.array(_df_shuffled_draws.iloc[:, i])
    return _df_shuffled_draws

# Queuing-as df first


In [244]:
df_customers = pd.DataFrame([i for i in range(3)], columns=['customer_id'])
df_customers['required_call_time'] = [random.randint(1,5) for i in range(3)]
df_customers['time_in']=[random.randint(1,10) for i in range(3)]
df_customers['time_out']=[0 for i in range(3)]
df_customers['time_assigned'] = [0 for i in range(3)]
df_customers['assigned_rep'] = ['none' for i in range(3)]
df_customers=df_customers.sort_values(by='time_in')
df_customers

,customer_id,required_call_time,time_in,time_out,time_assigned,assigned_rep
1,1,5,4,0,0,none
0,0,1,7,0,0,none
2,2,5,9,0,0,none


In [245]:
df_q = pd.DataFrame([0 for x in range(10)], columns=['num_enqued'])
df_q['update_event'] = [0 for x in range(10)]

df_reps = pd.DataFrame(columns=['rep1_busy', 'rep2_busy'])
df_reps['rep1_busy']=[0 for x in range(10)]
df_reps['rep2_busy']=[0 for x in range(10)]

In [246]:
for t in range(10):
    for customer in df_customers[(df_customers.time_assigned==0)&(df_customers.time_in==t)].customer_id:
        df_q['num_enqued'][t]+=1
    
    if df_q['num_enqued'][t]!=0:
        for customer_qued in range(df_q['num_enqued'][t]):
            assigned=False
            for rep in df_reps.columns:
                if df_reps[rep][t]==0:
                    df_customers.loc[df_customers.customer_id==customer, ['time_assigned', 'time_out',  'assigned_rep']] = [t, t+int(df_customers[df_customers.customer_id==customer]['required_call_time']) , rep]
                    
                    df_reps.loc[t: t + int(df_customers[df_customers.customer_id==customer]['required_call_time']) , rep] = 1
                    assigned=True
                    break
            if not assigned:
                df_q.loc[t+1, 'num_enqued']+=1
# Need something better to add or subract num enqued


In [247]:
df_customers

,customer_id,required_call_time,time_in,time_out,time_assigned,assigned_rep
1,1,5,4,9,4,rep1_busy
0,0,1,7,8,7,rep2_busy
2,2,5,9,14,9,rep2_busy


In [248]:
df_q

,num_enqued,update_event
0,0,0
1,0,0
2,0,0
3,0,0
4,1,0
5,0,0
6,0,0
7,1,0
8,0,0
9,1,0


In [241]:
df_reps

,rep1_busy,rep2_busy
0,0,0
1,0,0
2,1,0
3,1,1
4,1,1
5,1,0
6,0,0
7,0,0
8,0,0
9,0,0


,customer_id,required_call_time,time_in,time_out,time_assigned,assigned_rep
0,0,1,3,0,0,none
2,2,2,6,0,0,none
1,1,4,9,0,0,none


In [85]:
# new_customers = [np.random.choice([0,0,0,1]) for i in range(10)]
# df_customers = pd.DataFrame(new_customers, columns=['new_customer'])
# df_customers['required_call_time'] = df_customers.new_customer.apply(lambda x: random.randint(1,5) if x == 1 else 0)
# df_customers

# for t in range(10):
#     if df_customers.iloc[t, 0]==1:
#         df_q['update_event'][t]=1
        
#         assigned=False
#         for rep in df_reps.columns:
#             assigned=False
#             if df_reps[rep][t] == 0:
#                 df_reps[rep][t:t+df_customers.iloc[t,1]]=1
#                 df_q.iloc[t+df_customers.iloc[t,1]]=1
#                 break
#         if not assigned:
#             df_q['num_enqued'][t]+=1

# pd.concat([df_customers,df_reps,df_q], axis=1)